# L'*Anthologie* comme recueil de *textes* 

Avant toute chose, l'Anthologie est un recueil de textes. Commençons donc par nous intéresser à ceux-ci, et à comprendre comment ils se structurent. 

In [ ]:
import json
import requests
import csv
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
with open('dump_api.json','r') as f:
    data = json.loads(f.read()) 

Le dump de l'API est désormais contenu dans la variable ```data```. Ses données se structurent comme suit : 

In [ ]:
data[1]

Nous pouvons facilement vérifier cela en vérifiant la longueur de la liste : nous devrions obtenir 4134 résultats : le nombre d'épigrammes sur la plateforme. 

In [ ]:
print("La liste data contient",len(data), "entrées.")

Commençons par nous créer un dataframe ```Pandas``` contenant les valeurs suivantes : 

- id de l'épigramme 
- url de l'épigramme 
- url du texte 
- langue du texte 
- texte 

In [ ]:
df = pd.DataFrame(columns=['epi_id', 'epi_url', 'text_url', 'text_lang', 'text'])

In [ ]:
for item in data:
    texts = item['texts']
    for text in texts:
        df = pd.concat([df, pd.DataFrame({
            'epi_id': [item['id']],
            'epi_url': [item['url']],
            'text_url': [text['url']],
            'text_lang': [text['language']],
            'text': [text['text']]
        })], ignore_index=True)

pd.set_option('display.max_colwidth', None)

In [ ]:
df.head()

Nous pouvons normaliser quelque peu les textes avant de nous plonger dans une analyse de ceux-ci. Commençons par enlever les sauts de ligne et la ponctuation. La ponctuation pourrait être intéressante pour certaines analyses mais je ne m'y attarderai pas ici. 

In [ ]:
liste_ponctuation = [',', ';', ':', '.', '·', '!','?', '(', ')', '[', ']', '–', '«', '»']
liste_sauts = ['\n', '\r', '-', '\’', '\'']

def normalized(texte): 
    for caractere in liste_ponctuation:
        texte = texte.replace(caractere, '')
    for saut in liste_sauts:
        texte = texte.replace(saut, ' ')
    return texte

In [ ]:
df['text'] = df['text'].apply(normalized).str.lower()
df

De là, nous pouvons facilement compter le nombre de textes en grec, en anglais, italien, etc. 

In [ ]:
df['text_lang'].value_counts()

Ces données nous permettent de faire notre première visualisation ! Voyons, sous forme de tarte, la répartition des différentes langues au sein des textes de l'AG : 

In [ ]:
ax = df['text_lang'].value_counts().plot(kind='pie')
ax.set_title("Distribution des langues des textes de l'AGr")

Nous pouvons également compter le nombre de caractères et le nombre de mots dans chacun des textes : 

In [ ]:
df['len'] = df['text'].str.len()
df['words'] = df['text'].str.split().str.len()

In [ ]:
df[['epi_id', 'epi_url', 'text_url', 'text_lang', 'text', 'len', 'words']]

Voyons le nombre total de caractère pour chacune des langues : 

In [ ]:
sum_len = df.groupby('text_lang')['len'].sum()
sum_words = df.groupby('text_lang')['words'].sum()

In [ ]:
print(sum_len)
print(sum_words)

In [ ]:
ax1 = sum_len.plot(kind='pie')
ax.set_title("Nombre de caractère par langues au sein des textes de l'AGr")

In [ ]:
ax1 = sum_words.plot(kind='pie')
ax.set_title("Nombre de mots par langues au sein des textes de l'AGr")

On pourrait également calculer le nombre moyen de caractères et de mots par épigramme en fonction des langues : 

In [ ]:
mean_len = df.groupby('text_lang')['len'].mean()
print(mean_len)

mean_word = df.groupby('text_lang')['words'].mean()
print(mean_word)

In [ ]:
ax = mean_len.plot(kind='bar')
ax.set_title("Nombre moye de caractère par épigramme au sein de l'AGr selon les langues")

In [ ]:
ax = mean_word.plot(kind='bar')
ax.set_title("Nombre moye de mots par épigramme au sein de l'AGr selon les langues")